In [6]:
from model import LinUCB
from ipywidgets import interact, widgets, Output, HTML, Layout, VBox, HBox
from IPython.display import YouTubeVideo, HTML, IFrame 
from yaspin import yaspin

from random import randrange
import pandas as pd
import numpy as np
from utils import load_data, preprocess_fast, preprocess_without_title, preprocess_with_title, get_next_song_context
from urllib.parse import urlparse, parse_qs


# Load and preprocess data
data = load_data("data/songs.csv")

songs_links = pd.read_csv("data/songs_links.csv")

In [2]:
def get_youtube_id(link):
    return parse_qs(urlparse(link).query).get("v", [None])[0]

In [53]:
features = preprocess_fast(data)

# Initialize your LinUCB model
num_actions = len(features)
num_features = features.shape[1]
alpha = 0.5
linucb_model = LinUCB(num_actions=num_actions, num_features=num_features, alpha=alpha)
chosen_action = randrange(num_actions)

# Store user ratings
user_ratings = []

# Define interactive GUI elements
song_title_display = Output(layout=Layout(height='200px', width='200px', overflow_y='scroll'))  # Set the maximum height here

current_song = HTML(
    value=f"<b>Current song:  </b><i>{data.loc[chosen_action, 'Title']}</i>",
   )
youtube_display = Output(layout=Layout(height='500px'))  # Set the maximum height for YouTube video
rating_slider = widgets.IntSlider(min=1, max=10, description='Rating:')
suggest_button = widgets.Button(description='Rate Song')

# Initial display
initial_youtube_link = songs_links.loc[chosen_action, 'link']
with youtube_display:
    youtube_display.clear_output(wait=True)
    display(YouTubeVideo(get_youtube_id(initial_youtube_link), width=500, height=500))

with song_title_display:
    display(HTML(
    value="<h2>Ratings List</h2>",
   ))

# Define layout structure
left_column = VBox([current_song, song_title_display])
right_column = VBox([youtube_display, HBox([rating_slider, suggest_button])])
layout = HBox([left_column, right_column])

# Display the GUI
display(layout)

def suggest_song(b):
    global chosen_action

    suggest_button.disabled = True
    
    # Get current title
    title = data.loc[chosen_action, 'Title']
    
    # Get user rating
    rating = rating_slider.value

    # Store ratings (debug)
    user_ratings.append((title, rating))
    
    with song_title_display:
        print(f"{title}: {rating}/10")
        
    # Get the next song context
    _, context = get_next_song_context(data, features, chosen_action)

    
    # Choose the action (song) based on the model
    chosen_action = linucb_model.choose_action(context)

    current_song.value = f"<b>Current song:  </b><i>{data.loc[chosen_action, 'Title']}</i>"
        
    # Get new YouTube link
    youtube_link = songs_links.loc[chosen_action, 'link']

    # Suggest New video
    with youtube_display:
        youtube_display.clear_output(wait=True)
        display(YouTubeVideo(get_youtube_id(youtube_link), width=500, height=500, allow_autoplay=True))

    # Update the model with user feedback
    linucb_model.update_model(context=context, action=chosen_action, reward=rating)

    suggest_button.disabled = False

# Assign the suggest_song function to the button click event
suggest_button.on_click(suggest_song)